In [3]:
import pandas as pd
import requests as req
import numpy as np
import json
import time 
from lxml import etree
import os
import pickle as pkl
from pathlib import Path
from pandarallel import pandarallel
from tqdm import tqdm
tqdm().pandas()


0it [00:00, ?it/s]


In [4]:
output_path = Path.home() / "data/cetesb/"
estacoes_filepath = output_path / "estacoes-cetesb.csv.gz"

In [5]:
#URI do WebService
URI_estacoes = "https://servicos.cetesb.sp.gov.br/arcgis/rest/services/QUALAR/CETESB_QUALAR/MapServer/6/query"

# IDs das estações
objectIds = np.arange(1, 100)
objectIds = ["{}".format(i) for i in objectIds]
objectIds = ", ".join(objectIds)

In [6]:
# Jogar o pedido para o site
params = {"f": "json", "returnGeometry": "true",
          "outFields": "*", "objectIds": objectIds}
r = req.get(URI_estacoes, params)
json_response = json.loads(r.text)

# Processar os dados e gerar um dataset
dataset_estacoes = []
for feature in json_response["features"]:
    estacao_info = {}
    estacao_info.update(feature["attributes"])
    estacao_info.update(feature["geometry"])
    dataset_estacoes.append(estacao_info)
dataset_estacoes = pd.DataFrame(dataset_estacoes)

# Salvar o resultado
dataset_estacoes.to_csv(estacoes_filepath, compression="gzip", index=False)

In [7]:
# URL do site com as correspondências
URI_estacoes_2 = "http://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios.php"
r = req.get(URI_estacoes_2)
r.encoding = "utf-8"
raw_html = r.text
tree = etree.HTML(raw_html)
estacoes = tree.xpath("//select/option/@value")
estacoes_id2 = {}
for estacao in estacoes:
    (id2, name) = estacao.split("/")
    estacoes_id2[name] = id2
dataset_estacoes = pd.read_csv(estacoes_filepath, compression="gzip", na_values=["--"])
dataset_estacoes["ID2"] = None
for name, id2 in estacoes_id2.items():
    ind = (dataset_estacoes.Nome == name)
    dataset_estacoes.loc[ind, "ID2"] = id2    
    if (np.sum(ind) == 0):
        print("Não foi possível fazer a correspondência na estação {} ({})".format(name, id2))

ind = ~pd.isnull(dataset_estacoes.ID2)
dataset_estacoes = dataset_estacoes[ind]
dataset_estacoes.to_csv(estacoes_filepath, compression="gzip", index=False)

Não foi possível fazer a correspondência na estação S.José Campos-Vista Verde (57)


In [8]:
# URI do site com dados horários
URI_dados_horarios = "https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php"
URI_dados_horarios_meteo = "https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_Metrl.php"
# Carregar o arquivo com as informações sobre as estações
dataset_estacoes = pd.read_csv(estacoes_filepath, compression="gzip", na_values=["--"])

In [16]:
# Parâmetros livres
#inds = (dataset_estacoes.Municipio == "SAO PAULO")
#inds &= (dataset_estacoes.Nome == "Cerqueira César")
#inds &= (dataset_estacoes.Nome == "Pinheiros")
estacoes_id = dataset_estacoes.ID2.values

date_begin = "2013-05-09"
date_end = "2019-08-17"
date_range = pd.date_range(date_begin, date_end, freq="1d").sort_values(ascending=False)

In [17]:
def get_pollution_data(estacao_id, date):
    """
    Obtém a poluição na estação e data fornecidos
    """
    date_string = "{day:02d}/{month:02d}/{year:04d}".format(year=date.year, month=date.month, day=date.day)
    params = {"texData": date_string, "selEst": estacao_id}
    r = req.post(URI_dados_horarios, params)
    raw_html = r.text
    
    path = "{}_{}.html".format(estacao_id, date)
    
    with open(output_path / "raw_html" / path, 'w') as fid:
        fid.write(raw_html)

    return None

In [18]:
elements = pd.Series([(estacao_id, date) for date in date_range for estacao_id in estacoes_id])

In [ ]:
pollution_data = elements.progress_apply(lambda x: get_pollution_data(x[0], x[1]))

 67%|██████▋   | 93575/139812 [15:28:27<8:00:51,  1.60it/s]  

In [ ]:
pollution_data.to_pickle("aaa.pkl")

In [ ]:
len(pollution_data)